In [1]:
# menudisplay.ipynb
import sys
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QWidget, QVBoxLayout, QPushButton, QGridLayout, QMessageBox, QStackedWidget, QSpacerItem, QSizePolicy
)
from PyQt5.QtCore import Qt, QSettings
import pandas as pd
import zmq
from threading import Thread

class FoodOrderingWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.settings = QSettings("MyApp", "MyAppSettings")
        self.load_settings()
        self.setWindowTitle("سیستم سفارش غذا")
        self.setGeometry(100, 100, self.screen_width, self.screen_height)

        self.cart = []
        self.current_food_index = 0
        self.food_buttons = []

        self.stacked_widget = QStackedWidget()
        self.setCentralWidget(self.stacked_widget)

        self.food_selection_widget = QWidget()
        self.quantity_widget = QWidget()
        self.confirm_widget = QWidget()

        self.setup_food_selection_widget()
        self.setup_quantity_widget()
        self.setup_confirm_widget()

        self.stacked_widget.addWidget(self.food_selection_widget)
        self.stacked_widget.addWidget(self.quantity_widget)
        self.stacked_widget.addWidget(self.confirm_widget)

        self.load_food_data()

        # Setup ZMQ subscriber
        self.context = zmq.Context()
        self.socket = self.context.socket(zmq.SUB)
        self.socket.connect("tcp://localhost:5555")
        self.socket.setsockopt_string(zmq.SUBSCRIBE, '')
        self.running = True
        Thread(target=self.zmq_listener, daemon=True).start()

    def load_settings(self):
        self.screen_width = self.settings.value("screen_width", 1024, int)
        self.screen_height = self.settings.value("screen_height", 768, int)
        self.display_font_size = self.settings.value("display_font_size", 12, int)

    def setup_food_selection_widget(self):
        layout = QVBoxLayout()
        layout.setAlignment(Qt.AlignCenter)

        title = QLabel("منوی غذا")
        title.setAlignment(Qt.AlignCenter)
        title.setStyleSheet("font-size: 24px; font-weight: bold; margin-bottom: 20px; color: #2c3e50;")
        layout.addWidget(title)

        self.food_grid = QGridLayout()
        self.food_grid.setSpacing(20)
        layout.addLayout(self.food_grid)

        self.food_selection_widget.setLayout(layout)

    def setup_quantity_widget(self):
        self.quantity_layout = QVBoxLayout()
        self.quantity_layout.setAlignment(Qt.AlignCenter)

        self.quantity_label = QLabel()
        self.quantity_label.setAlignment(Qt.AlignCenter)
        self.quantity_label.setStyleSheet("font-size: 18px; margin-bottom: 20px; color: #2c3e50;")
        self.quantity_layout.addWidget(self.quantity_label)

        self.quantity_buttons = QVBoxLayout()
        self.add_quantity_buttons(range(1, 6))

        self.more_button = QPushButton("بیشتر")
        self.more_button.setStyleSheet("font-size: 18px; padding: 10px; background-color: #1abc9c; color: white;")
        self.more_button.clicked.connect(self.show_more_options)
        self.quantity_layout.addLayout(self.quantity_buttons)
        self.quantity_layout.addWidget(self.more_button)

        self.quantity_next_button = QPushButton("ادامه")
        self.quantity_next_button.setStyleSheet("font-size: 18px; padding: 10px; background-color: #3498db; color: white;")
        self.quantity_next_button.clicked.connect(self.confirm_or_continue)
        self.quantity_layout.addWidget(self.quantity_next_button)

        self.back_button_from_quantity = QPushButton("بازگشت")
        self.back_button_from_quantity.setStyleSheet("font-size: 18px; padding: 10px; background-color: #e74c3c; color: white;")
        self.back_button_from_quantity.clicked.connect(lambda: self.stacked_widget.setCurrentWidget(self.food_selection_widget))
        self.quantity_layout.addWidget(self.back_button_from_quantity)

        self.quantity_widget.setLayout(self.quantity_layout)

    def setup_confirm_widget(self):
        layout = QVBoxLayout()
        layout.setAlignment(Qt.AlignCenter)

        self.confirm_label = QLabel("آیا خرید را نهایی می‌کنید یا ادامه می‌دهید؟")
        self.confirm_label.setAlignment(Qt.AlignCenter)
        self.confirm_label.setStyleSheet("font-size: 18px; margin-bottom: 20px; color: #2c3e50;")
        layout.addWidget(self.confirm_label)

        self.finalize_button = QPushButton("نهایی کردن خرید")
        self.finalize_button.setStyleSheet("font-size: 18px; padding: 10px; background-color: #2ecc71; color: white;")
        self.finalize_button.clicked.connect(self.finalize_purchase)
        layout.addWidget(self.finalize_button)

        self.continue_button = QPushButton("ادامه خرید")
        self.continue_button.setStyleSheet("font-size: 18px; padding: 10px; background-color: #f39c12; color: white;")
        self.continue_button.clicked.connect(self.continue_shopping)
        layout.addWidget(self.continue_button)

        self.back_button_from_confirm = QPushButton("بازگشت")
        self.back_button_from_confirm.setStyleSheet("font-size: 18px; padding: 10px; background-color: #e74c3c; color: white;")
        self.back_button_from_confirm.clicked.connect(lambda: self.stacked_widget.setCurrentWidget(self.quantity_widget))
        layout.addWidget(self.back_button_from_confirm)

        self.confirm_widget.setLayout(layout)

    def load_food_data(self):
        try:
            df = pd.read_excel("food_data.xlsx")
            row, col = 0, 0
            for index, row_data in df.iterrows():
                button = QPushButton(f"{row_data['Food Name']} - ${row_data['Price']}")
                button.setStyleSheet("font-size: 18px; padding: 20px; background-color: #ecf0f1; color: #2c3e50;")
                button.clicked.connect(lambda checked, item=row_data: self.select_food(item))
                self.food_grid.addWidget(button, row, col)
                self.food_buttons.append(button)
                col += 1
                if col > 2:  # 3 columns
                    col = 0
                    row += 1
        except FileNotFoundError:
            QMessageBox.warning(self, "خطا", "فایل داده غذا پیدا نشد.")

    def select_food(self, item):
        self.selected_food = (item['Food Name'], item['Price'])
        self.quantity_label.setText(f"تعداد {self.selected_food[0]} را وارد کنید:")
        self.clear_more_options()
        self.stacked_widget.setCurrentWidget(self.quantity_widget)

    def add_quantity_buttons(self, range_values):
        for i in range_values:
            button = QPushButton(str(i))
            button.setStyleSheet("font-size: 18px; padding: 10px; background-color: #ecf0f1; color: #2c3e50;")
            button.clicked.connect(lambda checked, value=i: self.select_quantity(value))
            self.quantity_buttons.addWidget(button)

    def show_more_options(self):
        self.add_quantity_buttons(range(6, 12))
        self.more_button.setEnabled(False)

    def clear_more_options(self):
        for i in reversed(range(self.quantity_buttons.count())):
            widget = self.quantity_buttons.itemAt(i).widget()
            if widget is not None:
                widget.setParent(None)
        self.add_quantity_buttons(range(1, 6))
        self.more_button.setEnabled(True)

    def select_quantity(self, value):
        self.cart.append((self.selected_food[0], self.selected_food[1], value))
        self.stacked_widget.setCurrentWidget(self.confirm_widget)

    def confirm_or_continue(self):
        self.stacked_widget.setCurrentWidget(self.confirm_widget)

    def finalize_purchase(self):
        QMessageBox.information(self, "تایید خرید", "خرید شما نهایی شد!")
        self.cart = []
        self.stacked_widget.setCurrentWidget(self.food_selection_widget)

    def continue_shopping(self):
        self.stacked_widget.setCurrentWidget(self.food_selection_widget)

    def zmq_listener(self):
        while self.running:
            try:
                mode = int(self.socket.recv_string())
                self.handle_gesture(mode)
            except zmq.ZMQError as e:
                print(f"ZMQ Error: {e}")

    def handle_gesture(self, mode):
        if mode == 0:
            self.stacked_widget.setCurrentWidget(self.food_selection_widget)
        elif mode == 1:
            self.move_up()
        elif mode == 2:
            self.move_down()
        elif mode in [3, 4, 5, 6, 7]:
            self.select_food_by_mode(mode)
        elif mode == 8:
            self.go_back()

    def move_up(self):
        if self.stacked_widget.currentWidget() == self.food_selection_widget:
            self.current_food_index = (self.current_food_index - 1) % len(self.food_buttons)
            self.food_buttons[self.current_food_index].setFocus()
        elif self.stacked_widget.currentWidget() == self.quantity_widget:
            # Custom behavior for moving up in quantity selection
            pass
        elif self.stacked_widget.currentWidget() == self.confirm_widget:
            # Custom behavior for moving up in confirm screen
            pass

    def move_down(self):
        if self.stacked_widget.currentWidget() == self.food_selection_widget:
            self.current_food_index = (self.current_food_index + 1) % len(self.food_buttons)
            self.food_buttons[self.current_food_index].setFocus()
        elif self.stacked_widget.currentWidget() == self.quantity_widget:
            # Custom behavior for moving down in quantity selection
            pass
        elif self.stacked_widget.currentWidget() == self.confirm_widget:
            # Custom behavior for moving down in confirm screen
            pass

    def select_food_by_mode(self, mode):
        index = mode - 3
        if index < len(self.food_buttons):
            self.food_buttons[index].click()

    def go_back(self):
        current_widget = self.stacked_widget.currentWidget()
        if current_widget == self.quantity_widget:
            self.stacked_widget.setCurrentWidget(self.food_selection_widget)
        elif current_widget == self.confirm_widget:
            self.stacked_widget.setCurrentWidget(self.quantity_widget)

app = QApplication(sys.argv)
window = FoodOrderingWindow()
window.show()
app.exec_()


0